# RAG pipeline

In [40]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import FAISS
import pandas as pd
import psycopg2
from langchain_huggingface import HuggingFaceEmbeddings


In [41]:
# Step 4: Initialize LLaMA LLM via Ollama
# freeze
llm = Ollama(model="llama3.1:latest")
# llm = Ollama(model="llama3.1:70b")


In [42]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

bge_m3_ef = BGEM3EmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    device='cpu', # Specify the device to use, e.g., 'cpu' or 'cuda:0'
    use_fp16=False # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
)

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 383625.37it/s]
/Volumes/home/Learn/Master/3.LLMs/rag_pipeline/.venv/lib/python3.11/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue o

## Milvus standalone

In [43]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530"
)

In [44]:
# if client.has_collection(collection_name="demo_collection"):
#     client.drop_collection(collection_name="demo_collection")
# client.create_collection(
#         collection_name="demo_collection",
#         dimension=1024,  # The vectors we will use in this demo has 768 dimensions
# )


In [45]:
import os
import pandas as pd

# Đường dẫn đến thư mục "Khoi"
folder_path = '../Khoi'

# Danh sách lưu trữ các DataFrame
df_list = []

# Duyệt qua tất cả các file trong thư mục
# for file_name in os.listdir('Khoi'):
for file_name in ['Văn Bản Pháp Luật_p100000-100019.parquet']:
    # Kiểm tra nếu file có định dạng giống "Văn Bản Pháp Luật_p*.parquet"
    print("file_name", file_name)
    file_path = os.path.join(folder_path, file_name)
    
    # Đọc file Parquet và thêm vào danh sách DataFrame
    file_df = pd.read_parquet(file_path)
    
    # Thực hiện các chuyển đổi tương tự như bạn đã đề cập
    file_df['created_date'] = pd.to_datetime(file_df['created_date'], format='%d/%m/%Y')
    file_df['updated_date'] = pd.to_datetime(file_df['updated_date'], format='%d/%m/%Y')
    file_df['content_raw'] = file_df['content'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    
    # df_list.append(file_df)
    
    

    # connection = create_connection()

    # cursor = connection.cursor()

    try: 
        # Chèn dữ liệu từ DataFrame vào PostgreSQL
        
        data = []
        for index, item in file_df.iterrows():
            print(item)
            docs= [f"{item.title}"]
            
            docs_embeddings = bge_m3_ef.encode_documents(docs)

            vectors = docs_embeddings["dense"]

        
            data.append({"id": item.id, "vector": vectors[0], "text": docs[0],"url":item.url,"title": item.title, "subject": "van_ban_phap_luat"} )
        
        res = client.insert(collection_name="demo_collection", data=data)
        print(res)
        
        # Xác nhận các thay đổi vào cơ sở dữ liệu
        # connection.commit()

    except Exception as e:
        print(e)

    # Đóng kết nối
    # cursor.close()
    # connection.close()

    print("Data inserted successfully!")


# # Nối tất cả các DataFrame lại với nhau thành một DataFrame duy nhất
# combined_df = pd.concat(df_list, ignore_index=True)

# # Hiển thị vài dòng đầu tiên của DataFrame
# combined_df.head()

file_name Văn Bản Pháp Luật_p100000-100019.parquet
id                                                         100000
url             https://thuvienphapluat.vn/van-ban/Quyen-dan-s...
title           Quyết định 720/QĐ-CTN năm 2020 về cho thôi quố...
created_date                                  2020-05-18 00:00:00
updated_date                                  2020-06-03 00:00:00
content         b'<div class="content1">\n <div>\n  <div>\n   ...
content_raw     <div class="content1">\n <div>\n  <div>\n   <t...
Name: 0, dtype: object
id                                                         100001
url             https://thuvienphapluat.vn/van-ban/Quyen-dan-s...
title           Quyết định 719/QĐ-CTN năm 2020 về cho thôi quố...
created_date                                  2020-05-18 00:00:00
updated_date                                  2020-06-03 00:00:00
content         b'<div class="content1">\n <div>\n  <div>\n   ...
content_raw     <div class="content1">\n <div>\n  <div>\n   <t...
Na

In [46]:

res = client.query(
    collection_name="demo_collection",  # target collection
    filter='',  # number of returned entities
    limit=100,
    output_fields=["id", 'text'],  # specifies fields to be returned
    # sorted=True, #
)
for item in res:
    print(item)

In [47]:
queries = ["năm 2020 về cho thôi quốc tịch Việt Nam"]

query_embeddings = bge_m3_ef.encode_queries(queries)

res = client.search(
    collection_name="demo_collection",  # target collection
    data=[query_embeddings['dense'][0]],  # query vectors
    limit=5,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)
context_items = res[0]

print(context_items)



[{'id': 100000, 'distance': 0.6928851008415222, 'entity': {'text': 'Quyết định 720/QĐ-CTN năm 2020 về cho thôi quốc tịch Việt Nam đối với 95 công dân hiện đang cư trú tại Đức do Chủ tịch nước ban hành', 'subject': 'van_ban_phap_luat'}}, {'id': 100001, 'distance': 0.6902773976325989, 'entity': {'text': 'Quyết định 719/QĐ-CTN năm 2020 về cho thôi quốc tịch Việt Nam đối với 15 công dân hiện đang cư trú tại Hồng Kông (Trung Quốc) do Chủ tịch nước ban hành', 'subject': 'van_ban_phap_luat'}}, {'id': 100005, 'distance': 0.5462599396705627, 'entity': {'text': 'Quyết định 647/QĐ-TTg năm 2020 về phê duyệt Đề án hợp tác quốc tế về phát triển bền vững kinh tế biển Việt Nam đến 2030 do Thủ tướng Chính phủ ban hành', 'subject': 'van_ban_phap_luat'}}, {'id': 100002, 'distance': 0.4899764955043793, 'entity': {'text': 'Quyết định 648/QĐ-TTg năm 2020 về phê duyệt nhiệm vụ lập quy hoạch mạng lưới cơ sở y tế thời kỳ 2021-2030, tầm nhìn đến năm 2045 do Thủ tướng Chính phủ ban hành', 'subject': 'van_ban_pha

In [48]:
context = [item['entity']['text'] for item in context_items]
context 

['Quyết định 720/QĐ-CTN năm 2020 về cho thôi quốc tịch Việt Nam đối với 95 công dân hiện đang cư trú tại Đức do Chủ tịch nước ban hành',
 'Quyết định 719/QĐ-CTN năm 2020 về cho thôi quốc tịch Việt Nam đối với 15 công dân hiện đang cư trú tại Hồng Kông (Trung Quốc) do Chủ tịch nước ban hành',
 'Quyết định 647/QĐ-TTg năm 2020 về phê duyệt Đề án hợp tác quốc tế về phát triển bền vững kinh tế biển Việt Nam đến 2030 do Thủ tướng Chính phủ ban hành',
 'Quyết định 648/QĐ-TTg năm 2020 về phê duyệt nhiệm vụ lập quy hoạch mạng lưới cơ sở y tế thời kỳ 2021-2030, tầm nhìn đến năm 2045 do Thủ tướng Chính phủ ban hành',
 'Quyết định 13/2020/QĐ-UBND bãi bỏ Quyết định 10/2015/QĐ-UBND quy định về chính sách hỗ trợ doanh nghiệp xuất khẩu phần mềm trên địa bàn thành phố Đà Nẵng']

In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [50]:
prompt = '''
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question or history of the chat. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
History:{history}
Answer: 

'''

In [51]:
history = []

In [52]:
example_messages = llm.invoke(prompt)

In [53]:
example_messages

"I'm ready to help. What's the question?"

In [54]:
def question_llm(question):
  prompt_formatted = prompt.format(question=question, context=context, history=history)
  res = llm.invoke(prompt_formatted)
  history.append([
    ('user',question),
    ('system',res)
  ])
  
  # print(prompt_formatted)
  print(res)

In [55]:
question_llm("Quyết định 720/QĐ-CTN năm 2020 là gì")

Quyết định 720/QĐ-CTN năm 2020 là quyết định cho thôi quốc tịch Việt Nam đối với 95 công dân hiện đang cư trú tại Đức. Đây là một trong những quyết định của Chủ tịch nước về việc cho thôi quốc tịch Việt Nam. Quyết định này được ban hành vào năm 2020.


In [56]:
question_llm("Bạn lấy thông tin từ đâu")

Tôi lấy thông tin từ dữ liệu đã được cung cấp, bao gồm các quyết định của Chủ tịch nước và Thủ tướng Chính phủ năm 2020. Thông thường, tôi sẽ trả lời dựa trên dữ liệu đó.


In [57]:
question_llm("Câu hỏi về Quyết định 720/QĐ-CTN khi nãy bạn lấy thông tin từ dữ liệu nào")

Tôi lấy thông tin từ dữ liệu đã được cung cấp, bao gồm các quyết định của Chủ tịch nước và Thủ tướng Chính phủ năm 2020.


In [58]:
question_llm("Thể hiện các context mà bạn thao khảo")

Tôi không biết.


## FAQ
Khi nhận câu hỏi -> Hệ thống detect FAQ -> Trả lời FAQ answer

==> Người dùng sửa câu trả lời --> Lưu 

==> Nếu có version của user khác sửa rồi thì cho user đánh giá cái nào tốt hơn (random 2 cái)

Sau 1 khoảng tg câu trả lời nào điểm cao thì update vào FAQ

In [59]:
# client.drop_collection(collection_name="faq_collection")



In [60]:

client.create_collection(
        collection_name="faq_collection",
        dimension=1024,  # The vectors we will use in this demo has 768 dimensions
        auto_id=True
)

RPC error: [create_collection], <MilvusException: (code=65535, message=create duplicate collection with different parameters, collection: faq_collection)>, <Time:{'RPC start': '2024-09-23 15:14:19.419487', 'RPC error': '2024-09-23 15:14:19.427281'}>
Failed to create collection: faq_collection


MilvusException: <MilvusException: (code=65535, message=create duplicate collection with different parameters, collection: faq_collection)>

In [7]:
data = [
  {'question': 'Quyết định 720/QĐ-CTN năm 2020 là gì', 'answer': '1'},
  {'question': 'Quyết định 720/QĐ-CTN năm 2020 có gì', 'answer': '2'},
  {'question': 'Tôi yêu em', 'answer': '3'},
  ]
docs= [f"{item['question']}" for item in data ]
            
docs_embeddings = bge_m3_ef.encode_documents(docs)

vectors = docs_embeddings["dense"]

data = [{"question": data[i]['question'], "vector": vectors[i], "answer":  data[i]['answer']} for i in range(len(vectors)) ]

res = client.insert(collection_name="faq_collection", data=data)
res

{'insert_count': 3, 'ids': [452331441182736994, 452331441182736995, 452331441182736996], 'cost': 0}

In [22]:
faq_queries = ["Quyết định 720/QĐ-CTN năm 2020"]

faq_query_embeddings = bge_m3_ef.encode_queries(faq_queries)

faq_query_embeddings

{'dense': [array([-0.04193483,  0.00843451, -0.02539786, ..., -0.01222823,
         -0.06493812, -0.03544782], dtype=float32)],
 'sparse': <Compressed Sparse Row sparse array of dtype 'float64'
 	with 10 stored elements and shape (1, 250002)>}

In [23]:
res = client.search(
    collection_name="faq_collection",  # target collection
    data=[faq_query_embeddings['dense'][0]],  # query vectors
    limit=5,  # number of returned entities
    output_fields=["question", "answer"],  # specifies fields to be returned
)
context_items = res[0]

print(context_items)



[{'id': 1, 'distance': 0.9326093196868896, 'entity': {'question': 'Quyết định 720/QĐ-CTN năm 2020 có gì', 'answer': '2'}}, {'id': 0, 'distance': 0.9128784537315369, 'entity': {'question': 'Quyết định 720/QĐ-CTN năm 2020 là gì', 'answer': '1'}}, {'id': 2, 'distance': 0.2726913392543793, 'entity': {'question': 'Tôi yêu em', 'answer': '3'}}]
